<a href="https://colab.research.google.com/github/samarkinav/ds_project/blob/main/DSproject3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2
!pip install natasha
!pip install nltk

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 8.2MB 6.1MB/s 
     |████████████████████████████████| 34.4MB 123kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=2a3e5885ab8c70230cfbdbd13ed34a8992fb1ad53b7382de66b06cbc076d1d9d
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [2]:
import nltk
import pymorphy2
import pandas as pd
import re
import numpy as np
import collections

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC' -O Mosru_dump_all_pages.txt

--2021-04-26 16:52:23--  https://docs.google.com/uc?export=download&id=1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC
Resolving docs.google.com (docs.google.com)... 216.58.192.78, 2607:f8b0:4026:800::200e
Connecting to docs.google.com (docs.google.com)|216.58.192.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1654bgobmt7mhenthl2iiiriakefddmg/1619455875000/03619448018617981983/*/1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC?e=download [following]
--2021-04-26 16:52:25--  https://doc-10-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1654bgobmt7mhenthl2iiiriakefddmg/1619455875000/03619448018617981983/*/1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC?e=download
Resolving doc-10-ak-docs.googleusercontent.com (doc-10-ak-docs.googleusercontent.com)... 172.217.3.1, 2607:f8b0:4026:800::2001
Connecting to doc-10-ak-docs.googleusercontent.com (doc-10-ak-docs.googleuser

In [4]:
df = pd.read_csv('Mosru_dump_all_pages.txt', engine='python', sep="', '", names=['тема','текст','что-то','ответственный','дата'])
df.head()

,тема,текст,что-то,ответственный,дата
0,'Сообщение о проблеме «Несоблюдение требований...,Выше второго этажа,**********,начальник Объединения административно-техниче...,16 Ноября 2020 в 16:31'
1,'Сообщение о проблеме «Неубранная городская те...,По адресу Снежная д24 расположена музыкальная ...,**********,глава управы района Свиблово города Москвы,05 Мая 2020 в 15:45'
2,'Сообщение о проблеме «Неубранная городская те...,После проведения работ на кабельной канализаци...,**********,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57'
3,'Сообщение о проблеме «Неубранная городская те...,Очистите опору освещения. Приведите в надлежащ...,**********,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54'
4,'Сообщение о проблеме «Захламление территории»...,Более двух недель лежит куча грунта в перемешк...,**************,и.о. главы управы района Свиблово города Москвы,24 Августа 2019 в 15:39'


In [5]:
del df['что-то']

In [6]:
df.isna().sum()

тема             0
текст            0
ответственный    0
дата             0
dtype: int64

Мы хотим вытащить из последнего столбца даты. Сделаем это с помощью библиотеки Natasha. 

In [7]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,

    DatesExtractor, 

    Doc)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

dates_extractor = DatesExtractor(morph_vocab)

Функция обработки дат:



In [8]:
def preparing_dates(string):
    matches = dates_extractor(string)
    facts = [_.fact.as_json for _ in matches]

    for f in facts:
        date = (f"{f.get('day')}.{f.get('month')}.{f.get('year')}")
        year = (f"{f.get('year')}")
    return [date, year]

Обрабатывать адреса будем с помощью регулярных выражений. Используя их же, вычленим из первого столбца саму жалобу.

In [9]:
address_list = []
appeal_list = []
year_list = []
for i in range(df.shape[0]):
    [df.iloc[i, 3], year] = preparing_dates(df.iloc[i, 3])
    year_list.append(year)

    address_list.append(re.search('по адресу (.*)', df.iloc[i, 0]).group(1))
    appeal_list.append(re.search('«(.*)»', df.iloc[i, 0], flags=re.IGNORECASE).group(1))

df['адрес'] = address_list
df['тема'] = appeal_list
df['год']= year_list


In [10]:
df.head(20)

,тема,текст,ответственный,дата,адрес,год
0,Несоблюдение требований к размещению информаци...,Выше второго этажа,начальник Объединения административно-техниче...,16.11.2020,Снежная улица д.26,2020
1,Неубранная городская территория,По адресу Снежная д24 расположена музыкальная ...,глава управы района Свиблово города Москвы,5.5.2020,Снежная улица д.24Д,2020
2,Неубранная городская территория,После проведения работ на кабельной канализаци...,начальник Объединения административно-техниче...,1.3.2020,Снежная улица д.6,2020
3,Неубранная городская территория,Очистите опору освещения. Приведите в надлежащ...,глава управы района Свиблово города Москвы,16.1.2020,Снежная улица д.24Д,2020
4,Захламление территории,Более двух недель лежит куча грунта в перемешк...,и.о. главы управы района Свиблово города Москвы,24.8.2019,Снежная улица д.1А,2019
5,Захламление территории,"Осколки бордюрного камня спрятаны за дерево, в...",глава управы района Свиблово города Москвы,5.5.2019,Снежная улица д.12 с.2,2019
6,Несоблюдение требований к размещению информаци...,Прошу рассмотреть законность размещения вывеск...,глава управы района Свиблово города Москвы,6.2.2019,Снежная улица д.26,2019
7,Несоблюдение требований к размещению информаци...,Просьба проверить законность размещения информ...,глава управы района Свиблово города Москвы,5.10.2018,Снежная улица д.26,2018
8,Несоблюдение требований к размещению информаци...,Здравствуйте! Прошу проверить законность устан...,глава управы района Свиблово города Москвы,22.3.2018,Снежная улица д.23,2018
9,Отсутствие информационного щита,Отсутствуют информационные щиты на всех подобн...,генеральный директор Государственного бюджетн...,5.9.2017,Снежная улица д.6,2017


Мы хоти вытащить из 3 столбца название района, в котором зарегистрирована проблема. Для этого нам нужна функция приведения слова к нормальной форме:

In [11]:
test_df = df

In [12]:
def to_norm(x):
  morph = pymorphy2.MorphAnalyzer()
  p = morph.parse(x)[0]
  return p.normal_form

In [ ]:
districts = []
for i in range(test_df.shape[0]):
  if re.search('управы (.*) района', test_df.iloc[i, 2]):
    district = re.search('управы (.*) района', test_df.iloc[i, 2]).group(1)
  elif re.search('района (.*) города', test_df.iloc[i, 2]):
    district = (re.search('района (.*) города', test_df.iloc[i, 2]).group(1))
  elif re.search('района (.*)', test_df.iloc[i, 2]):
    district = (re.search('района (.*)', test_df.iloc[i, 2]).group(1))
  else:
    district = np.nan
  try:
    district = district.replace('»','')
    district = district.replace('"','')
    district = district.strip()
    district = to_norm(district)
  except:
    pass
  districts.append(district)

test_df['район'] = districts

In [ ]:
test_df

Многие обращения не содержат названия районов. Попробуем создать некий "словарь" формата улица-район

In [ ]:
df_street_district = pd.DataFrame(columns=['street','district'])

streets = []
new_districts = []

for i in range(test_df.shape[0]):
  try:
    test_df.iloc[i, 6].isdigit()
    streets.append(re.search('(.*) улица', test_df.iloc[i, 4], flags=re.IGNORECASE).group(1))
    new_districts.append(test_df.iloc[i, 6])
  except:
    pass

df_street_district['district'] = new_districts
df_street_district['street'] = streets
df_street_district.drop_duplicates(inplace=True, ignore_index=True)
df_street_district

Заметим, что есть одинаковые названия улиц в разных районов. Попробуем заполнить пропуски районов, опираясь на этот словарь. Алгоритм такой: берем название улицы, ищем её в словаре, если найдено одно совпадение, то записываем в колонку "район" соответствующий район из словаря; если не найдено соответсвий или найдено больше одного - не записываем ничего

In [ ]:
for i in range(test_df.shape[0]):
  for j in range(df_street_district.shape[0]):
    try:
      test_df.iloc[i, 6].isdigit()
    except:
      try:
        street = re.search('(.*) улица', test_df.iloc[i, 4], flags=re.IGNORECASE).group(1)
        if street == df_street_district.iloc[j, 0] and collections.Counter(df_street_district.iloc[:,0])[street] == 1:
          test_df.iloc[i, 6] = df_street_district.iloc[j, 1]
      except:
        pass

In [ ]:
test_df.head(200)

In [ ]:
test_df['дата'] = pd.to_datetime(test_df['дата'])

Теперь воспользуемся скрытым размещением Дирихле для тематетического моделирования. Это удобный способ выявления тем, присутствующих в корпусе, а также проявляющихся в разных сочетаниях в каждом документе в корпусе.

In [ ]:
!pip install gensim
!pip install pyLDAvis

Функция обработки текста включает в себя процесс стемминга (нахождение основы слова)

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer

russian_stopwords = stopwords.words('russian')
regex = re.compile('[%s]' % re.escape(string.punctuation))
stemmer = SnowballStemmer('russian')

def preprocessing(text):
  text = regex.sub('', text)
  text = [token for token in text.split() if token not in russian_stopwords]
  text = [stemmer.stem(token) for token in text]
  text = [token for token in text if token]
  return ' '.join(text)


In [ ]:
for i in range(test_df.shape[0]):
  test_df['текст'] = test_df['текст'].apply(lambda x: preprocessing(x))

Для улучшения выделения тематик, необходимо выделить коллокации (словосочетания, имеющие признаки синтаксически или семантически целостной единицы). Алгоритм выделения коллокаций реализован в библиотеки Gensim. Для его работы переведем наш корпус из коллекции документов, в коллекцию документов - токенов.

In [ ]:
text_clean = []
for index, row in test_df.iterrows():
  text_clean.append(row['текст'].split())

from gensim.models import Phrases
bigram = Phrases(text_clean)
trigram = Phrases(bigram[text_clean])

for idx in range(len(text_clean)):
  for token in bigram[text_clean[idx]]:
    if '_' in token:
      text_clean[idx].append(token)
  for token in trigram[text_clean[idx]]:
    if '_' in token:
      text_clean[idx].append(token)

Теперь в нашем корпусе есть устойчивые выражения, и они не будут рассмотрены как отдельные слова, тем самым повысилась устойчивость корпуса.
Следующим шагом преобразуем наш корпус в словарь частот.

In [ ]:
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(text_clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)
corpus = [dictionary.doc2bow(doc) for doc in text_clean]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

Создание LDA модели:

In [ ]:
from gensim.models.ldamulticore import LdaMulticore
model = LdaMulticore(corpus=corpus, id2word = dictionary, num_topics=4)
model.show_topics()

In [ ]:
import pyLDAvis.gensim_models
import gensim
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

Теперь подберем самый важный параметр - количество тем. Для этого нам понадобится такое понятие, как когерентность. Тема называется когерентной (или согласованной), если термины, наиболее частые в данной теме, неслучайно часто соместно встречаются рядом в документах коллекции. Для оценивания когерентности будем использовать поточечную взаимную информацию (pointwise mutual information, PMI)

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
  coherence_values = []
  model_list = []
  for num_topics in range(start, limit, step):
    model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics)
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())
  return [model_list, coherence_values]


model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=text_clean, start=2, limit=40, step=2)

Для понимания согласованности визуализируем как количество тем влияют на согласованность, создадим график при помощи пакета Matplotlib

In [ ]:
import matplotlib.pyplot as plt
limit=40; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel('Количество тем')
plt.ylabel('Согласованность')
plt.legend(('coherence_values'), loc='best')
plt.show()